<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/STS/BERTNextSentenceModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 가져오기 시도

데이터 증식은 성공!

In [ ]:
!pip install sentencepiece transformers torch

In [ ]:
!pip install seqeval
!pip install torchtext pytorch-lightning
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install sentence_transformers numpy scikit-learn scipy nltk tqdm

In [1]:
import pytorch_lightning as pl
pl.__version__

'1.5.10'

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
from transformers import BertConfig, BertModel, AdamW

In [4]:
from tqdm import tqdm, tqdm_notebook
from typing import Callable, List, Tuple
from seqeval.metrics import f1_score, accuracy_score

In [5]:
import torch
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

In [6]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Colab Notebooks/NextLab/news_class9x1400'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
def text_splicing(text, max_len) :
  before_idx = 0
  li = []
  i = 0
  while True:
    if before_idx+max_len < len(text) :
      li.append( [text[before_idx:before_idx+max_len], i] )
      i += 1
    else :
      li.append(  [text[before_idx:], i] )
      if len(li) < 2 :
        return None   # 연결할 단락이 없으므로 dataset에 넣지 않는다.
      return li

    before_idx += max_len
  return li

In [8]:
max_len = 128

In [9]:
text = """특정한 풍경을 통하여 개인적인 감정을 시각화한 사진전
박진호는 1990년대에 자신의 벗은 몸을 복사기로 복제하여 실험적인 결과물을 생산한 작가로서 유명하다.
그런데 이번에는 새벽녘의 달과 하늘을 카메라 앵글에 담아서 감상적인 이미지를 전시하였다.
작가가 이번에 인사동에 있는 나우 갤러리에서 전시한 작품들은 달빛과 구름 낀 새벽하늘을 감성적인 느낌이 드는 결과물로 재구성하여 보여주고 있다.
전시 작품마다 외형적으로 컬러가 자극적이고 전체적으로 톤도 어두워서 보는 이들을 감성적으로 동화시키는 표현전략을 구사하고 있다는 것을 알 수 있다.
작가가 관심을 갖고 카메라 앵글에 담은 대상을 구체적으로 살펴보면 구름과 산봉우리 그리고 강변 풍경이다.
그런데 왠지 표현대상과 소재가 낯설게 느껴지지 않고 너무나도 익숙하다.
왜 그런 것 일까?
그것은 작가가 표현대상으로 선택한 소재와 표현방식이 ‘일요 사진가’라고 일컬어지는 아마추어 작가들이 흔히 찍는 탐미적인 사진과 별다른 차이점이 없기 때문이다.
사진은 시각예술이다.
그러므로 작가가 표현하고자하는 주제와 관계없이 외형적으로 보여 지고 느껴지는 것이 무엇보다도 중요하다.
그런데도 아쉽게도 이번에 박진호가 발표한 풍경사진들은 외형적으로 아마추어 작가들의 유미주의적이고 감상적인 작품과의 차별화에 실패하였다.
풍경사진이나 정물사진은 절제된 프레이밍과 세련된 컬러와 톤이 작품의 완성도를 보장하는데 있어서 중요한 요소로 작용한다.
그러므로 주제선택과 더불어서 외형적으로는 그것을 좀 더 염두에 두고 작업을 진행을 했다면 최종 결과물의 완성도가 달라졌을 것이다.
이번에 작가가 발표한 작품들은 누구나 폭넓게 이해할 수 있는 소재와 표현방식을 보여주고 있다. 하지만 자신만의 조형언어를 보여주는 데는 소홀히 했다는 것이 느껴진다.
그래서 여러 가지로 아쉬운 점이 많은 전시가 되었다. 하지만 자신의 감정을 솔직하게 표현하여 결과물을 생산 한 것은 분명하다.
특정한 풍경을 통하여 개인적인 감정을 시각화 전시이다.""".replace('\n', '')

text_splicing(text, max_len) 


[['특정한 풍경을 통하여 개인적인 감정을 시각화한 사진전박진호는 1990년대에 자신의 벗은 몸을 복사기로 복제하여 실험적인 결과물을 생산한 작가로서 유명하다.그런데 이번에는 새벽녘의 달과 하늘을 카메라 앵글에 담아서 감상적인 이미지를',
  0],
 [' 전시하였다.작가가 이번에 인사동에 있는 나우 갤러리에서 전시한 작품들은 달빛과 구름 낀 새벽하늘을 감성적인 느낌이 드는 결과물로 재구성하여 보여주고 있다.전시 작품마다 외형적으로 컬러가 자극적이고 전체적으로 톤도 어두워서 보는 ',
  1],
 ['이들을 감성적으로 동화시키는 표현전략을 구사하고 있다는 것을 알 수 있다.작가가 관심을 갖고 카메라 앵글에 담은 대상을 구체적으로 살펴보면 구름과 산봉우리 그리고 강변 풍경이다.그런데 왠지 표현대상과 소재가 낯설게 느껴지지 않고 ',
  2],
 ['너무나도 익숙하다.왜 그런 것 일까?그것은 작가가 표현대상으로 선택한 소재와 표현방식이 ‘일요 사진가’라고 일컬어지는 아마추어 작가들이 흔히 찍는 탐미적인 사진과 별다른 차이점이 없기 때문이다.사진은 시각예술이다.그러므로 작가가 ',
  3],
 ['표현하고자하는 주제와 관계없이 외형적으로 보여 지고 느껴지는 것이 무엇보다도 중요하다.그런데도 아쉽게도 이번에 박진호가 발표한 풍경사진들은 외형적으로 아마추어 작가들의 유미주의적이고 감상적인 작품과의 차별화에 실패하였다.풍경사진이',
  4],
 ['나 정물사진은 절제된 프레이밍과 세련된 컬러와 톤이 작품의 완성도를 보장하는데 있어서 중요한 요소로 작용한다.그러므로 주제선택과 더불어서 외형적으로는 그것을 좀 더 염두에 두고 작업을 진행을 했다면 최종 결과물의 완성도가 달라졌을',
  5],
 [' 것이다.이번에 작가가 발표한 작품들은 누구나 폭넓게 이해할 수 있는 소재와 표현방식을 보여주고 있다. 하지만 자신만의 조형언어를 보여주는 데는 소홀히 했다는 것이 느껴진다.그래서 여러 가지로 아쉬운 점이 많은 전시가 되었다. 하',
  6],
 ['지만 자신의 감정을 솔직하게 표

In [ ]:
import os
content = []
for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
              label = path[path.rindex('/')+1:]
              text = f.read()
              temp_list = text_splicing(text.replace('\n', ' '), max_len)
              if temp_list is not None :
                content.extend(temp_list)

len(content), len(content[0]) # content : [(text, 단락 번호)]

In [11]:
len(content)

78845

In [12]:
# label : 0 is True, 1 is False
import random
i, len_content = 0, len(content)
dataset = []
right_li, wrong_li = [], []   # 추가할 list
before_text = content[0][0]
while i < len_content :
  text = content[i][0]
  idx = content[i][1]
  if idx > 0 :    # label : right
    right_li.append( (before_text, text, 0) )   # right

  # idx == 0 and len(li) == 1 인 경우는 없다. file read할 때 넣지 않았다. 
  elif idx == 0 and len(right_li) > 1 :   # label : wrong 
    len_li = len(right_li)
    for t in right_li :
      rd = random.randint(0, len_content-len_li-1)
      wrong_text = content[rd][0] if rd < i-len_li else content[rd+len_li][0]   # random으로 고르기 ( 같은 기사 제외 )
      wrong_li.append((t[0], wrong_text, 1) )
    dataset.extend(right_li)
    dataset.extend(wrong_li)
    right_li, wrong_li = [], []
  before_text = text
  i += 1
len_content, len(dataset)

(78845, 149252)

In [13]:
dataset[-3:]

[('임원들 보수 총액이 아닌 임원 개인별 보수가 공시되도록 했다. 이 의원은 18개 국내은행이 정부로부터 대외채무 지급보증을 받으면서 은행장의 연봉 삭감 결의를 했는데 과연 삭감내역이 적정한지 알기 어렵기 때문에 각 임원별 공시를 도',
  ' 되면 P씨의 부모님 부양 부담이 많이 덜어질 수 있기 때문이다. 여유가 있다면 자녀에게 국민연금을 물려주는 것도 좋은 선물이 된다. 18세 이상의 자녀를 국민연금에 임의가입시키고 그 돈을 부모님이 내주는 것이다. 이 경우 부모님',
  1),
 ('입할 필요가 있다고 말했다. 또한 이 의원은 이전부터 제기되어온 문제인 지배주주가 보수결정을 좌우하고 임원의 독립성을 침해하는 문제도 임원별로 보수가 공시되어야 방지될 수 있다며 개정안이 통과되면 책임경영은 물론이고 임원 보수와 ',
  '들 간에 마찰이 빚어졌다. 주민들과의 마찰로 인해 측량이 불가능하다고 판단한 공사 직원들이 측량을 포기하고 돌아가기로 결정하면서 이날 일은 거기서 일단락되는 듯했다. 하지만 사무실로 돌아가겠다던 공사 직원들은 길을 우회하여 강정천',
  1),
 ('업무 간의 상관성도 향상시켜줄 것이라고 말했다. 이 의원은 임원의 독립성을 강화하고 회사의 지배주주가 임원 보수의 명목으로 우회배당을 하거나 회사 재산을 처분하는 등 회사 재산의 사용에서 사익만을 추구하는 행동을 억제하고, 적정성',
  '세 규모는 63조8000억 원이며 대기업과 고소득층의 감세 혜택분은 31조 원(48.6%), 중소기업·중산서민층은 32.5조 원(50.9%)이라고 발표했다. 하지만 국회예산정책처(NABO)의 자료를 토대로 하면 문 후보의 말이 옳',
  1)]

In [14]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train, test = train_test_split(dataset[:1000], test_size=0.2, random_state=0, shuffle=True)
train, val = train_test_split(train, test_size=0.2, random_state=0, shuffle=True)
print("train :", len(train), "  val :", len(val), "  test :", len(test))

train : 640   val : 160   test : 200


In [15]:
del content
del dataset
del right_li
del wrong_li

In [16]:
train[0]

('일 먼저 항공방제를 거부했다. 항공방제의 특성상 어느 한 곳만 빼고 농약을 할 수 없기 때문이었다. 이로 인해 주변 사람들한테 눈총도 많이 받았다. 주민들의 심정은 충분히 이해를 하죠. 과수 농사는 한 번 망치면 끝이거든요. 성공',
 '당 대전시당 관계자는 이윤호 장관의 의중을 잘 안다, 정책보좌관에게 확인했더니 (대전시장 출마설은) 전혀 사실이 아니라고 얘기하고 있다고 전했다. 이 관계자는 현 시장이 당연히 공천 받는 것은 아니고 다른 분이 출마한다고 하면 경',
 1)

# 여기까지 ok

In [ ]:
# https://github.com/jhgan00/ko-sentence-transformers
# from sentence_transformers import SentenceTransformer, util
# import numpy as np

# embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

# model_path = '/content/drive/My Drive/Colab Notebooks/NextLab/nli/result.pt'
# from sentence_transformers import SentenceTransformer, util
# import numpy as np
# from transformers import BertModel, DistilBertModel
# distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

In [18]:
# pytorch, transformers
from transformers import BertForNextSentencePrediction

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)

model = BertForNextSentencePrediction.from_pretrained("snunlp/KR-Medium")

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
import math
# 이어지는 두 개의 문장
prompt = " 메카로 불리는 걸까. 암베르크 공장은 커다란 병원 수술"
next_sentence = "실 같았다. 안내자가 “건초 더미에서 바늘을 찾는 게 더 쉬울 것”이라고 말할 정도로 먼지 한점 없이 깨끗했다."
encoding = tokenizer(prompt, next_sentence, return_tensors='pt', padding=True )    # 두 문장을 이어붙여서 encode
print(type(encoding))
print(encoding)
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
print(logits)
softmax = torch.nn.Softmax()
probs = softmax(logits)
print(probs)
print('최종 예측 레이블 :', torch.argmax(probs))

<class 'transformers.tokenization_utils_base.BatchEncoding'>
{'input_ids': tensor([[    2,  2850,  5363,  5016, 17396,  1965,  5045,    18,  3525,  5467,
         14118, 11525,  5019,  4198,  5042,  5109, 10016, 12728,     3,  3389,
          1936,  8660,    18, 13544,  8573,   211,  1963,  5817,  2436,  5031,
          8454,  2955,  5946,  5008, 16657,  1977,  2436,  3349,  5340,  1969,
           212,  9118, 13555, 10845, 14089,  4494,  5450,  9043, 10269,  8468,
            18,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}
tensor([[ 1.5886, -0.8314]], grad_fn=<AddmmBackward0>)
tensor([[0.9183, 0.0817]], grad_fn=<Softm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


In [51]:
tokenizer.decode([2, 3])

'[CLS] [SEP]'

In [65]:
tokenizer.decode(0), tokenizer.decode(1)  # unk : 없는 글자, unknown, pad 문장의 길이를 맞추기위함

('[ P A D ]', '[ U N K ]')

In [ ]:
# from tokenization_kobert import KoBertTokenizer
# tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

In [ ]:
# model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)

+ Korean RoBERTa
+ KorSTS 
+ Kr-bert-base

SentenceTransformer : 문장, 텍스트, 이미지 임베딩, cosine 유사성과 비교하여 유사한 의미를 가진 문장 찾기

In [81]:
from transformers import BertTokenizer, BertForNextSentencePrediction, BertConfig
import torch

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertForNextSentencePrediction.from_pretrained("bert-base-uncased")
# prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
# next_sentence = "The sky is blue due to the shorter wavelength of blue light."
# encoding = tokenizer(prompt, next_sentence, return_tensors="pt")
# outputs = model(**encoding, labels=torch.LongTensor([1]))
# logits = outputs.logits
# assert logits[0, 0] < logits[0, 1]  # next sentence was random

In [82]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

## CorpusDataset

In [99]:
class CorpusDataset(Dataset):
    def __init__(self, sentences, transform: Callable):
        self.sentences = sentences
        self.transform = transform

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        prompt = self.sentences[idx][0]
        next_sentence = self.sentences[idx][1]
        label = self.sentences[idx][2]
        (
            input_ids,
            attention_mask,
            token_type_ids,
            label, 
        ) = self.transform(prompt, next_sentence, label)

        return input_ids, attention_mask, token_type_ids, label

In [79]:
tok_prompt = tokenizer.tokenize(prompt)    # 두 문장을 이어붙여서 encode
tok_next_sen = tokenizer.tokenize(next_sentence)    # 두 문장을 이어붙여서 encode

input_ids = [2] + tokenizer.convert_tokens_to_ids(tok_prompt) + [3]    # 처음과 끝 표시
input_ids_next_sen = [2]+ tokenizer.convert_tokens_to_ids(tok_next_sen)+ [3]

slicing_idx = 0
if len(input_ids) + len(input_ids_next_sen) > max_len :
  slicing_idx = len(input_ids) + len(input_ids_next_sen) - max_len // 2 + 2
  input_ids = [2] + input_ids[slicing_idx:]
  input_ids_next_sen = input_ids_next_sen[:-slicing_idx] + [3]

token_type_ids = [0]*len(input_ids)
token_type_ids.extend([1]*len(input_ids_next_sen))
input_ids.extend(input_ids_next_sen)
attention_mask = [1] * len(token_type_ids)
pad_length = max_len-len(input_ids)

input_ids.extend([0] * pad_length)
token_type_ids.extend([0] * pad_length) # pad : 0
attention_mask.extend([0] * pad_length) # pad : 0

53 53 53


(128, 128, 128)

## Preprocessor

In [110]:
from typing import List, Tuple
# encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

class Preprocessor :
    def __init__(self, max_len: int):
        self.tokenizer = BertTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(self, prompt, next_sentence, label
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """문장과 띄어쓰기 tagging에 대해 feature로 변환한다.

        Args:
            sentence: 문장
            tags: 띄어쓰기 tagging

        Returns:
            feature를 리턴한다.
            input_ids, attention_mask, token_type_ids, slot_labels
        """
        tok_prompt = tokenizer.tokenize(prompt)    # 두 문장을 이어붙여서 encode
        tok_next_sen = tokenizer.tokenize(next_sentence)    # 두 문장을 이어붙여서 encode

        input_ids = [2] + tokenizer.convert_tokens_to_ids(tok_prompt) + [3]    # 처음과 끝 표시
        input_ids_next_sen = [2]+ tokenizer.convert_tokens_to_ids(tok_next_sen)+ [3]

        slicing_idx = 0
        if len(input_ids) + len(input_ids_next_sen) > max_len :
          slicing_idx = len(input_ids) + len(input_ids_next_sen) - max_len // 2 + 2
          input_ids = [2] + input_ids[slicing_idx:]
          input_ids_next_sen = input_ids_next_sen[:-slicing_idx] + [3]

        token_type_ids = [0]*len(input_ids)
        token_type_ids.extend([1]*len(input_ids_next_sen))
        input_ids.extend(input_ids_next_sen)
        attention_mask = [1] * len(token_type_ids)
        pad_length = max_len-len(input_ids)

        input_ids.extend([0] * pad_length)
        token_type_ids.extend([0] * pad_length) # pad : 0
        attention_mask.extend([0] * pad_length) # pad : 0

        input_ids = torch.tensor(input_ids, dtype=torch.int)
        attention_mask = torch.tensor(attention_mask, dtype=torch.int)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.int)

        label = [1.0, 0.] if label == 0 else [0., 1.0]
        label = torch.tensor(label, dtype=torch.float)

        return input_ids, attention_mask, token_type_ids, label

## BERTNextSentenceModel

In [171]:

from math import log
class BertOnlyNSPHead(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.seq_relationship = nn.Linear(config.hidden_size, 2)

    def forward(self, pooled_output):
        seq_relationship_score = self.seq_relationship(pooled_output)
        return seq_relationship_score


class BERTNextSentenceModel(pl.LightningModule):
    def __init__(self, config, dataset):
        super().__init__()
        print("init")
        self.config = config
        self.dataset = dataset
        self.slot_labels_type = [0, 1]
        self.pad_token_id = 0
        self.softmax = torch.nn.Softmax()
        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=2
        )
        self.model = BertModel.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.cls = BertOnlyNSPHead(config)
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.slot_labels_type)
        )

    def forward(self,
        input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None,
        head_mask=None, inputs_embeds=None, labels=None, output_attentions=None, output_hidden_states=None,
        return_dict=None, **kwargs, ):
        """
        return NextSentencePredictorOutput(
            loss=next_sentence_loss,
            logits=seq_relationship_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

        """
        print("forward")
        logits = self.model(input_ids, token_type_ids=token_type_ids)[0]
        # print(logits)
        probs = self.softmax(logits)
        print(probs)
        # return torch.relu(self.linear(probs.view(probs.size(0), -1)))
        # output = self.model(input_ids, attention_mask, token_type_ids, position_ids, 
        #                        head_mask, inputs_embeds, labels,
        #                        output_attentions, output_hidden_states, return_dict, **kwargs,)
        return probs

    def training_step(self, batch, batch_nb):
        print("training_step")

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=slot_label_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels
        f1 = self._f1_score(outputs, slot_label_ids)
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        tensorboard_logs = {'train_loss': loss, 'train_f1':f1, 'train_acc':acc}
        return {"loss": loss, "f1": f1, "acc": acc, "log": tensorboard_logs}

    def training_end(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids) # slot_labels : labels
        f1 = self._f1_score(outputs, slot_label_ids)
        acc = self._calculate_accuracy(outputs, slot_label_ids)

        tensorboard_logs = {'train_loss': loss, 'train_f1':f1, 'train_acc':acc}
        print("training_end : ", tensorboard_logs)
        return {"loss": loss, "f1": f1, "acc": acc, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        super().on_train_epoch_end()
        print("training_epoch_end")
      

    def validation_step(self, batch, batch_nb):

        input_ids, attention_mask, token_type_ids, slot_label_ids = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_label_ids)
        val_f1 = self._f1_score(slot_label_ids, outputs)
        val_acc = self._calculate_accuracy(outputs, slot_label_ids)
        return {"val_loss": loss, "val_f1": val_f1, 'val_acc':val_acc}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        # val_f1 = torch.stack([x['val_f1'] for x in outputs]).mean()
        val_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss, 'val_acc':val_acc}
        print('validation_epoch_end : ', tensorboard_logs)
        self.log("validation_epoch_end : tensorboard_logs ", tensorboard_logs)

        return {'val_acc':val_acc, 'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def validation_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()
        # val_f1 = torch.stack([x["val_f1"] for x in outputs]).mean()
        tensorboard_logs = {
            "val_loss": val_loss,
            "val_acc" : val_acc
        }
        print("validation_end : ", tensorboard_logs)
        return {'val_acc':val_acc, 'val_loss': val_loss, 'log': tensorboard_logs}
    
    def test_step(self, batch, batch_nb):
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        # f1 = self._f1_score(gt_slot_labels, pred_slot_labels)
        acc = self._calculate_accuracy(outputs, slot_label_ids)
        loss = self._calculate_loss(outputs, slot_label_ids)
        return {"test_loss": loss, "test_acc": acc}

    def test_end(self, outputs):
        test_f1 = torch.stack([x["test_f1"] for x in outputs]).mean()
        test_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()
        self.log("test_loss", test_loss)
        self.log("test_acc", test_acc)
        self.log("test_f1", test_f1)
        print('test_end')
        return {"pred_slot_labels" : [x["pred_slot_labels"] for x in outputs], "test_loss": test_loss, "test_f1": test_f1, "test_acc": test_acc}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        f1 = torch.stack([x['test_f1'] for x in outputs]).mean()
        acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss, 'test_f1':f1, 'test_acc':acc}
        print('test_epoch_end : ', tensorboard_logs)
        self.log("test_epoch_end : tensorboard_logs ", tensorboard_logs)
        return {'test_acc':acc, 'test_loss': avg_loss, 'test_f1':f1, 'log': tensorboard_logs}

    def predict_step(self, batch, batch_idx, dataloader_idx=0):   # prediction : forward(), predict_step()
        input_ids, attention_mask, token_type_ids, slot_label_ids = batch    # slot_label은 없음.
        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pred_slot_labels, gt_slot_labels = self._convert_ids_to_labels(
            outputs, slot_label_ids
        )
        return {'pred_slot_labels':pred_slot_labels, 'gt_slot_labels': gt_slot_labels}


    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return DataLoader(self.dataset["train"], batch_size=self.config.eval_batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset["val"], batch_size=self.config.eval_batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset["test"], batch_size=self.config.eval_batch_size)

    def pred_dataloader(self, dataset):
        return DataLoader(dataset, batch_size=1)

    def _calculate_loss(self, outputs, labels):
        # active_logits = outputs.view(-1, len(self.slot_labels_type))
        # active_labels = labels.view(-1)
        print("outputs",outputs)
        print("outputs[3]",outputs[3])
        print("labels",labels)
        loss = F.cross_entropy(outputs[3], labels)
        return loss
        
    def _calculate_accuracy(self, outputs, labels):
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        accuracy = accuracy_score(active_logits, active_labels)
        return accuracy

    def _f1_score(self, gt_slot_labels, pred_slot_labels):
        return torch.tensor(
            f1_score(gt_slot_labels, pred_slot_labels), dtype=torch.float32
        )

    def _convert_ids_to_labels(self, outputs, slot_labels):
        _, y_hat = torch.max(outputs, dim=2)
        y_hat = y_hat.detach().cpu().numpy()
        slot_label_ids = slot_labels.detach().cpu().numpy()

        slot_label_map = {i: label for i, label in enumerate(self.slot_labels_type)}
        slot_gt_labels = [[] for _ in range(slot_label_ids.shape[0])]
        slot_pred_labels = [[] for _ in range(slot_label_ids.shape[0])]

        for i in range(slot_label_ids.shape[0]):
            for j in range(slot_label_ids.shape[1]):
                if slot_label_ids[i, j] != self.pad_token_id:
                    slot_gt_labels[i].append(slot_label_map[slot_label_ids[i][j]])
                    slot_pred_labels[i].append(slot_label_map[y_hat[i][j]])

        return slot_pred_labels, slot_gt_labels

## Config

In [172]:
# yaml 파일 대신에 객체로 생성
from transformers import BertModel, BertConfig
class Config(BertConfig):
  def __init__(self) :
    super().__init__()
    self.task= 'kor_nextsentence_prediction_20220322'
    self.log_path= data_path+'/logs'
    self.bert_model = "snunlp/KR-Medium"    # 수정해야함!
    self.max_len= 256
    self.train_batch_size= 128
    self.eval_batch_size= 128
    self.dropout_rate= 0.1
    self.gpus= torch.cuda.device_count()
config = Config()

# Train

In [173]:
preprocessor = Preprocessor(config.max_len)

In [174]:
dataset = {}
dataset['train'] = CorpusDataset(train, preprocessor.get_input_features)
dataset['val'] = CorpusDataset(val, preprocessor.get_input_features)
dataset['test'] = CorpusDataset(test, preprocessor.get_input_features)

In [175]:
len(dataset['train']),len(dataset['train'][0]), dataset['train'][0]

(640,
 4,
 (tensor([    2,  3749,  8951, 11769,  5112, 10042, 10714,  8468,    18, 11769,
           5112, 14540,  4374,  5331,  5139,  8979,  4494,  2016,  5079, 10290,
           2323, 14777,  4497,  3320,  3577,  5056,  9199,  9876,    18, 19030,
          10999,  9538,  8869,  8531,  2343,  5571,  5067,  8648, 11070,    18,
          10337,  8545,  3393,  8697, 11205, 17719,  4492,  5344,    18,  2023,
           5124,  2323,  8834,  4494,  2995, 18755,  5093, 13739, 15380,    18,
           9657,     3,     2,  2411, 12828,  5091,  5300, 10959,  3742,  5655,
           5147,  8882,  5105,  3737,  5213,  5008,  3771, 12029,    16,  9000,
           5187,  5083, 15796,  5054,  9337,  5222,  8696,    12, 12828,  9512,
          12379,  5221,  5019,    13,  9648, 11302, 10128,  9086,  8455,  9157,
          11193,    18,  3742, 10959,  4559, 16431,  9559, 11733, 10027,  8924,
           8742,  8590, 12549, 12379,  8808,  8775,  1998,     3,     0,     0,
              0,     0,     0,

In [176]:
# model = BertForNextSentencePrediction.from_pretrained(model_path)
model = BERTNextSentenceModel(config, dataset)

init


Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [177]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor

logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=3, name=config.task
)

acc_checkpoint_callback = ModelCheckpoint(
    dirpath=data_path+'/sts/checkpoints/'+ config.task, 
    filename="acc_{epoch}_{val_loss:35f}",
    verbose=True,
    monitor='val_acc',
    mode='max',
    save_top_k=3,
    save_last=True)

loss_checkpoint_callback = ModelCheckpoint(
    dirpath=data_path+'/sts/checkpoints/'+ config.task, 
    filename="val_{epoch}_{val_acc:.3f}",
    verbose=True,
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_last=True)

loss_early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=3,
    verbose=True,
    mode='min'
)

acc_early_stop_callback = EarlyStopping(
    monitor='val_acc',
    min_delta=0.0001,
    patience=3,
    verbose=True,
    mode='max'
)
lrmonitor_callback = LearningRateMonitor(logging_interval='step')

In [178]:
trainer = pl.Trainer(
    gpus=config.gpus,
    callbacks=[acc_checkpoint_callback, loss_checkpoint_callback, lrmonitor_callback],
    logger=logger,
    max_epochs=2,
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name    | Type            | Params
--------------------------------------------
0 | softmax | Softmax         | 0     
1 | model   | BertModel       | 101 M 
2 | cls     | BertOnlyNSPHead | 1.5 K 
3 | dropout | Dropout         | 0     
4 | linear  | Linear          | 1.5 K 
--------------------------------------------
101 M     Trainable params
0         Non-trainable params
101 M     Total params
405.618   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


forward


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[[0.0126, 0.0256, 0.0067,  ..., 0.0094, 0.0073, 0.0043],
         [0.0111, 0.0260, 0.0022,  ..., 0.0103, 0.0103, 0.0031],
         [0.0107, 0.0259, 0.0032,  ..., 0.0049, 0.0042, 0.0009],
         ...,
         [0.0080, 0.0068, 0.0073,  ..., 0.0065, 0.0018, 0.0021],
         [0.0084, 0.0081, 0.0085,  ..., 0.0059, 0.0025, 0.0021],
         [0.0113, 0.0105, 0.0075,  ..., 0.0133, 0.0069, 0.0046]],

        [[0.0078, 0.0050, 0.0034,  ..., 0.0037, 0.0039, 0.0087],
         [0.0042, 0.0073, 0.0023,  ..., 0.0061, 0.0097, 0.0058],
         [0.0045, 0.0054, 0.0063,  ..., 0.0047, 0.0084, 0.0085],
         ...,
         [0.0057, 0.0097, 0.0036,  ..., 0.0102, 0.0026, 0.0032],
         [0.0067, 0.0071, 0.0034,  ..., 0.0113, 0.0036, 0.0032],
         [0.0110, 0.0076, 0.0070,  ..., 0.0064, 0.0075, 0.0083]],

        [[0.0083, 0.0063, 0.0053,  ..., 0.0032, 0.0021, 0.0119],
         [0.0096, 0.0090, 0.0062,  ..., 0.0052, 0.0047, 0.0121],
         [0.0049, 0.0119, 0.0040,  ..., 0.0101, 0.0034, 0.

RuntimeError: ignored

In [ ]:
# bert model에 forward return 값 고쳐야함. : linear? relu??
# https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#bertfornextsentenceprediction